# BLACKCOFFER TEXT ANALYSIS

SUBMITTED BY: RITULAGNA TRIPATHY

**Importing Libraries**

In [714]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords

**SENTIMENT ANALYSIS**

In [1]:
#Creating a list of given stopwords

stopword_files = ["StopWords\StopWords_Auditor.txt", "StopWords\StopWords_Currencies.txt",
                "StopWords\StopWords_DatesandNumbers.txt", "StopWords\StopWords_Generic.txt",
                "StopWords\StopWords_GenericLong.txt","StopWords\StopWords_Geographic.txt","StopWords\StopWords_Names.txt"]
with open("stopwords.txt", "w") as new_created_file:
    for name in stopword_files:
        with open(name) as file:
            for line in file:
               new_created_file.write(line)

stopwords_given = []
data = open('stopwords.txt', 'r').read()
data = data.replace("\n"," ")
stopwords_given = data.lower().split()

In [716]:
#Cleaning using stopwords list
def clean_stopwords(dataset,stopwords_given):
   data = ' '.join([word for word in dataset.split() if word.lower() not in stopwords_given])
   return data

**Creating a dictionary of Positive & Negative words**

In [717]:
pos_words = open('MasterDictionary\positive-words.txt','r', encoding="utf8").read()
pos_words = pos_words.replace("\n"," ")
pos_words = pos_words.split(" ")

In [718]:
neg_words = open("MasterDictionary/negative-words.txt",'r').read()
neg_words = neg_words.replace("\n"," ")
neg_words = neg_words.split(" ")

In [719]:
output_data = pd.read_excel("Output Data Structure.xlsx")
output_data.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38,https://insights.blackcoffer.com/what-if-the-c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39,https://insights.blackcoffer.com/what-jobs-wil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40,https://insights.blackcoffer.com/will-machine-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41,https://insights.blackcoffer.com/will-ai-repla...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [720]:
pos_score = []
neg_score = []
pol_score = []
sub_score = []
avg_sen_len = []
percentage_complex_words = []
fog_index = []
avg_words_per_sen = []
count_complex_words = []
word_count = []
syllable_per_word = []
personal_pronouns = []
avg_word_len = []

In [721]:
#Calculation of different scores

for i in range(114):

    url_id = output_data._get_value(i,'URL_ID')
    file_path = f"{int(url_id)}.txt"
    dataset = open(file_path,'r',encoding='utf8').read()
    dataset = clean_stopwords(dataset,stopwords_given)
    words_dataset = dataset.split(" ")
    
    positive_score = 0
    negative_score = 0
    total_words_after_cleaning = len(words_dataset)

    for i in range (len(words_dataset)):
        if words_dataset[i] in pos_words:
            positive_score +=1
        elif words_dataset[i] in neg_words:
            negative_score +=1
    

    polarity_score = (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score)/ ((total_words_after_cleaning) + 0.000001)

    pos_score.append(round(positive_score))
    neg_score.append(round(negative_score))
    pol_score.append(round(polarity_score,2))
    sub_score.append(round(subjectivity_score,2))

   


**ANALYSIS OF READABILITY**


In [722]:
#Data Cleaning

#nltk stopwords
stopwords = open('nltk stopwords.txt','r', encoding='utf8').read()
stopwords = stopwords.split()

def clean_data(dataset):
    dataset = re.sub('[^a-zA-Z]', ' ', dataset) #To remove non-alphabetical characters
    dataset = re.sub(r'\s+', ' ', dataset) #To remove more than one blank space
    words_dataset = dataset.lower().split(" ")
    return words_dataset

In [723]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = ["a","e","i","o","u","y"]
    exceptions = ("es","ed","e")
    
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
                count += 1
        if word.endswith(exceptions):
            count -= 1
        if count == 0:
            count += 1
    return count


In [725]:
for i in range(114):

    url_id = output_data._get_value(i,'URL_ID')
    file_path = f"{int(url_id)}.txt"
    dataset = open(file_path,'r',encoding='utf8').read()

    sen_dataset = dataset.split(".")
    total_sen = len(sen_dataset) #Counting total no of sentences

    dataset = re.sub(r'[^\w\s]', ' ', dataset) #To remove non-alphanumeric characters
    dataset = re.sub(r'\s+', ' ', dataset) #To remove more than one blank space
    words_dataset = dataset.split(" ")
    
    total_words = len(words_dataset) #Counting total no of words

    #Count of personal pronouns
    pronouns = ['I','we','We','my','My','ours','our','Our','us']
    count_pronoun = 0

    #Calculation of average word length
    character_count = 0

    for word in words_dataset:
        character_count += len(word)
        if word in pronouns:
            count_pronoun += 1

    average_word_len = character_count/total_words
    avg_word_len.append(round(average_word_len))

    personal_pronouns.append(count_pronoun)

    #Clean word count
    clean_dataset = []
    for word in words_dataset:
        if word not in stopwords:
            clean_dataset.append(word)
    
    word_count.append(len(clean_dataset))

    #Calculation of no of complex words & count of syllables per word
    syllables = 0
    s_count = 0
    complex_words = 0

    for i in range(total_words):
        syllables = syllable_count(words_dataset[i])
        if (syllables>2):
            complex_words += 1

        s_count+=syllables
    
    count_complex_words.append(complex_words)

    syllable_count_per_word = s_count/total_words
    syllable_per_word.append(round(syllable_count_per_word))

    #Calculation of percentage of complex words
    percentage_of_complex_words = (complex_words/total_words)*100
    percentage_complex_words.append(round(percentage_of_complex_words))

    #Calculation of average words per sentence
    average_words_per_sen = total_words/total_sen
    avg_words_per_sen.append(round(average_words_per_sen))

    #Calculation of average sentence length
    average_sentence_length = average_words_per_sen
    avg_sen_len.append(round(average_sentence_length))

    #Calculation of fog index
    fog_i = (average_sentence_length+percentage_of_complex_words)*0.4
    fog_index.append(round(fog_i))


In [726]:
#Entering all the calculated data to the dataframe

output_data['POSITIVE SCORE']=pos_score
output_data['NEGATIVE SCORE']=neg_score
output_data['POLARITY SCORE']=pol_score
output_data['SUBJECTIVITY SCORE']=sub_score
output_data['AVG SENTENCE LENGTH']=avg_sen_len
output_data['PERCENTAGE OF COMPLEX WORDS']=percentage_complex_words
output_data['FOG INDEX']=fog_index
output_data['AVG NUMBER OF WORDS PER SENTENCE']=avg_words_per_sen
output_data['COMPLEX WORD COUNT']=count_complex_words
output_data['SYLLABLE PER WORD']=syllable_per_word
output_data['WORD COUNT']=word_count
output_data['PERSONAL PRONOUNS']=personal_pronouns
output_data['AVG WORD LENGTH']=avg_word_len

output_data

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,54,25,0.37,0.08,24,21,18,24,386,1234,2,6,6
1,38,https://insights.blackcoffer.com/what-if-the-c...,45,28,0.23,0.11,21,13,13,21,190,845,1,10,5
2,39,https://insights.blackcoffer.com/what-jobs-wil...,58,29,0.33,0.10,20,19,16,20,332,1083,1,5,5
3,40,https://insights.blackcoffer.com/will-machine-...,38,14,0.46,0.07,19,13,13,19,216,974,1,24,5
4,41,https://insights.blackcoffer.com/will-ai-repla...,40,18,0.38,0.07,22,15,15,22,270,1071,1,20,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,22,22,0.00,0.09,17,19,14,17,182,625,1,11,5
110,147,https://insights.blackcoffer.com/the-future-of...,28,6,0.65,0.06,21,16,15,21,182,733,1,4,5
111,148,https://insights.blackcoffer.com/big-data-anal...,27,33,-0.10,0.09,16,17,13,16,198,740,1,4,5
112,149,https://insights.blackcoffer.com/business-anal...,24,2,0.85,0.08,22,22,18,22,135,380,2,2,6


In [727]:
output_data.to_excel("Output.xlsx")